In [2]:
import os
os.chdir('../')

In [22]:
%pwd

'/Users/jinayshah/Desktop/Text_Summaizer'

In [23]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig :
    root_dir : Path
    source_url : str
    local_data_file : Path
    unzip_dir : Path 

In [24]:
from textsummarizer.constants import *
from textsummarizer.utils.common import read_yaml,create_dir

In [25]:
class ConfigurationManager :
    def __init__(self,config_path = CONFIGURE_FILE_PATH,param_path = PARAM_FILE_PATH):

        self.config = read_yaml(config_path)
        self.param = read_yaml(param_path)
        create_dir([self.config.artifacts_root])

    def get_ingestion_config(self)->DataIngestionConfig :
        config = self.config.data_ingestion
        create_dir([config.root_dir])
        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_url=config.source_url,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir
        )
        return data_ingestion_config

In [26]:
import os
import urllib.request as request
import zipfile
from textsummarizer.logging import logger
from textsummarizer.utils.common import get_size
 

In [27]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config

    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, header = request.urlretrieve(
                url=self.config.source_url,
                filename=self.config.local_data_file
            )
            logger.info(f'{filename} downloaded with following info: {header}')
        else:
            filename = self.config.local_data_file
            logger.info(f'{filename} already exists of size {get_size(Path(filename))}')

    def extract_zipfile(self):
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as file:
            file.extractall(unzip_path)


In [34]:
import os
import ssl
import logging
import zipfile
from pathlib import Path
from urllib import request

import certifi
from textsummarizer.utils.common import get_size

logger = logging.getLogger(__name__)

class DataIngestion:
    def __init__(self, config:DataIngestionConfig):
        self.config = config

    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            # Create an SSL context with the certifi CA certificates
            ssl_context = ssl.create_default_context(cafile=certifi.where())
            try:
                # Use urlopen with the SSL context to download the file
                with request.urlopen(self.config.source_url, context=ssl_context) as response:
                    with open(self.config.local_data_file, 'wb') as file:
                        file.write(response.read())
                logger.info(f'{self.config.local_data_file} downloaded successfully')
            except Exception as e:
                logger.error(f'Error during file download: {e}')
                raise
        else:
            filename = self.config.local_data_file
            logger.info(f'{filename} already exists of size {get_size(Path(filename))}')

    def extract_zipfile(self):
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as file:
            file.extractall(unzip_path)
        logger.info(f'ZIP file extracted to {unzip_path}')


In [33]:
try :
    config = ConfigurationManager()
    data_ingestion_config = config.get_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zipfile()
except Exception as e:
    raise e

[2024-01-30 23:47:47,747 : INFO : common : <_io.TextIOWrapper name='config/config.yaml' mode='r' encoding='UTF-8'> loaded]
[2024-01-30 23:47:47,750 : INFO : common : <_io.TextIOWrapper name='params.yaml' mode='r' encoding='UTF-8'> loaded]
[2024-01-30 23:47:47,751 : INFO : common : created a directory at : artifacts]
[2024-01-30 23:47:47,752 : INFO : common : created a directory at : artifacts/data_ingestion]
[2024-01-30 23:47:49,841 : INFO : 3016555923 : artifacts/data_ingestion/data.zip downloaded successfully]
[2024-01-30 23:47:49,935 : INFO : 3016555923 : ZIP file extracted to artifacts/data_ingestion]
